In [16]:
!pip install dgl

In [17]:
%matplotlib inline
import os

os.environ["DGLBACKEND"] = "pytorch"
import dgl
import numpy as np
import networkx as nx
import torch
import torch.nn as nn
import dgl.function as fn
import torch.nn.functional as F
import shutil
from torch.utils.data import DataLoader
import cloudpickle
from dgl.nn import GraphConv

In [18]:
current_dir = "./"
checkpoint_path = current_dir + "save_models/model_checkpoints/" + "checkpoint"
os.makedirs(checkpoint_path, exist_ok=True)

best_model_path = current_dir + "save_models/best_model/"

folder_data_temp = current_dir +"data_temp/"
shutil.rmtree(folder_data_temp, ignore_errors=True)

path_save = current_dir + "sider.zip"
shutil.unpack_archive(path_save, folder_data_temp)

In [19]:
""" Classification Dataset """
class DGLDatasetClass(torch.utils.data.Dataset):
    def __init__(self, address):
            self.address=address+".bin"
            self.list_graphs, train_labels_masks_globals = dgl.load_graphs(self.address)
            num_graphs =len(self.list_graphs)
            self.labels = train_labels_masks_globals["labels"].view(num_graphs,-1)
            self.masks = train_labels_masks_globals["masks"].view(num_graphs,-1)
            self.globals = train_labels_masks_globals["globals"].view(num_graphs,-1)
    def __len__(self):
        return len(self.list_graphs)
    def __getitem__(self, idx):
        return  self.list_graphs[idx], self.labels[idx], self.masks[idx], self.globals[idx]

In [20]:
path_data_temp = folder_data_temp + "scaffold"+"_"+str(0)
train_set = DGLDatasetClass(address=path_data_temp+"_train")
val_set = DGLDatasetClass(address=path_data_temp+"_val")
test_set = DGLDatasetClass(address=path_data_temp+"_test")

print(len(train_set), len(val_set), len(test_set))


1141 142 144


this code snippet loads the train, validation, and test datasets, concatenates them into a single dataset, and then analyzes the graph properties such as the number of vertices, edges, and graphs. It also computes and prints the shape of the adjacency matrices for each graph.

In [58]:
# Get the train, validation, and test datasets
train_set = DGLDatasetClass(address=path_data_temp + "_train")
val_set = DGLDatasetClass(address=path_data_temp + "_val")
test_set = DGLDatasetClass(address=path_data_temp + "_test")

# Concatenate the datasets
dataset = torch.utils.data.ConcatDataset([train_set, val_set, test_set])

# Get all the graphs in the dataset
graphs = [data[0] for data in dataset]

# Get the number of vertices, edges, and graphs
num_vertices = [graph.number_of_nodes() for graph in graphs]
num_edges = [graph.number_of_edges() for graph in graphs]
num_graphs = len(dataset)

print("Number of vertices:", num_vertices)
print("Number of edges:", num_edges)
print("Number of graphs:", num_graphs)

# Get the adjacency matrix shape
adj_matrix = [graph.adjacency_matrix() for graph in graphs]
adj_shape = [matrix.shape for matrix in adj_matrix]

print("Adjacency matrix shape:", adj_shape)


Number of vertices: [9, 1, 7, 3, 5, 9, 2, 7, 1, 4, 6, 1, 1, 1, 6, 5, 1, 7, 3, 5, 3, 3, 3, 3, 4, 3, 4, 1, 1, 5, 1, 1, 1, 1, 3, 1, 1, 1, 28, 5, 9, 5, 7, 3, 22, 6, 3, 18, 4, 2, 2, 1, 5, 5, 3, 3, 7, 56, 9, 7, 6, 5, 7, 6, 4, 5, 2, 4, 1, 3, 3, 12, 3, 15, 4, 5, 2, 13, 12, 9, 7, 4, 1, 7, 1, 7, 9, 2, 1, 1, 2, 8, 6, 3, 19, 3, 8, 15, 16, 2, 4, 7, 6, 3, 4, 4, 5, 13, 10, 2, 2, 13, 5, 7, 4, 11, 4, 1, 1, 7, 1, 2, 11, 3, 5, 34, 5, 32, 9, 7, 5, 1, 3, 5, 4, 1, 9, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 2, 2, 2, 8, 3, 6, 8, 6, 36, 4, 3, 6, 5, 3, 5, 17, 10, 4, 6, 9, 5, 4, 7, 4, 4, 5, 10, 8, 6, 8, 5, 11, 4, 4, 2, 5, 8, 4, 6, 6, 12, 8, 3, 7, 10, 4, 3, 4, 4, 8, 10, 5, 2, 2, 6, 17, 15, 17, 9, 17, 16, 6, 3, 3, 8, 4, 6, 7, 5, 10, 8, 4, 9, 8, 6, 9, 4, 7, 2, 2, 5, 3, 3, 5, 5, 7, 7, 5, 4, 9, 2, 9, 4, 7, 5, 10, 6, 4, 4, 3, 6, 8, 3, 9, 7, 20, 7, 8, 2, 6, 4, 5, 23, 15, 7, 6, 13, 12, 10, 7, 6, 6, 14, 4, 29, 7, 7, 9, 17, 7, 2, 31, 14, 13, 10, 7, 17, 6, 98, 6, 2, 2, 3, 3, 15, 6, 5, 11, 5, 7, 2, 13, 15, 10, 7, 13, 9

This  code focuses on analyzing the properties of a single graph rather than analyzing properties for all graphs in the dataset. It retrieves the first graph, calculates the number of vertices and edges, and computes the shape of the adjacency matrix.

In [59]:
# Get the train, validation, and test datasets
train_set = DGLDatasetClass(address=path_data_temp + "_train")
val_set = DGLDatasetClass(address=path_data_temp + "_val")
test_set = DGLDatasetClass(address=path_data_temp + "_test")

# Concatenate the datasets
dataset = torch.utils.data.ConcatDataset([train_set, val_set, test_set])

# Get the first graph in the dataset
graph = dataset[0][0]

# Get the number of vertices, edges, and graphs
num_vertices = graph.number_of_nodes()
num_edges = graph.number_of_edges()
num_graphs = len(dataset)

print("Number of vertices:", num_vertices)
print("Number of edges:", num_edges)
print("Number of graphs:", num_graphs)

# Get the adjacency matrix shape
adj_matrix = graph.adjacency_matrix()
adj_shape = adj_matrix.shape

print("Adjacency matrix shape:", adj_shape)


Number of vertices: 9
Number of edges: 16
Number of graphs: 1427
Adjacency matrix shape: (9, 9)


In this code snippet, the cumulative counts of vertices and edges are calculated for all graphs in the dataset. Additionally, the shape of the adjacency matrix is determined

In [60]:
# Get the train, validation, and test datasets
train_set = DGLDatasetClass(address=path_data_temp + "_train")
val_set = DGLDatasetClass(address=path_data_temp + "_val")
test_set = DGLDatasetClass(address=path_data_temp + "_test")

# Concatenate the datasets
dataset = torch.utils.data.ConcatDataset([train_set, val_set, test_set])

# Get the total number of graphs
num_graphs = len(dataset)

# Initialize variables to store cumulative counts
total_vertices = 0
total_edges = 0
adj_shape = None

# Iterate over all graphs in the dataset
for i in range(num_graphs):
    graph = dataset[i][0]  # Get the graph
    num_vertices = graph.number_of_nodes()  # Get the number of vertices
    num_edges = graph.number_of_edges()  # Get the number of edges

    # Update cumulative counts
    total_vertices += num_vertices
    total_edges += num_edges

    if adj_shape is None:
        adj_matrix = graph.adjacency_matrix()  # Get the adjacency matrix
        adj_shape = adj_matrix.shape  # Get the shape of the adjacency matrix

# Print the results
print("Number of vertices:", total_vertices)
print("Number of edges:", total_edges)
print("Adjacency matrix shape:", adj_shape)
print("Number of graphs:", num_graphs)


Number of vertices: 16601
Number of edges: 30988
Adjacency matrix shape: (9, 9)
Number of graphs: 1427


In this code snippet, the focus is on a single dataset rather than concatenating multiple datasets.
The code focuses on a single dataset and extracts information from the first graph in that dataset, including the number of vertices, edges, and the shape of the adjacency matrix. The results are then printed.

In [61]:
# Get the dataset
dataset = DGLDatasetClass(address=path_data_temp + "_train")

# Get the first graph in the dataset
graph = dataset[0][0]

# Get the number of vertices, edges, and graphs
num_vertices = graph.number_of_nodes()
num_edges = graph.number_of_edges()
num_graphs = len(dataset)

print("Number of vertices:", num_vertices)
print("Number of edges:", num_edges)
print("Number of graphs:", num_graphs)

# Get the adjacency matrix shape
adj_matrix = graph.adjacency_matrix()
adj_shape = adj_matrix.shape

print("Adjacency matrix shape:", adj_shape)



Number of vertices: 9
Number of edges: 16
Number of graphs: 1141
Adjacency matrix shape: (9, 9)


In [62]:
# Get the dataset
dataset = DGLDatasetClass(address=path_data_temp + "_val")

# Get the first graph in the dataset
graph = dataset[0][0]

# Get the number of vertices, edges, and graphs
num_vertices = graph.number_of_nodes()
num_edges = graph.number_of_edges()
num_graphs = len(dataset)

print("Number of vertices:", num_vertices)
print("Number of edges:", num_edges)
print("Number of graphs:", num_graphs)

# Get the adjacency matrix shape
adj_matrix = graph.adjacency_matrix()
adj_shape = adj_matrix.shape

print("Adjacency matrix shape:", adj_shape)


Number of vertices: 7
Number of edges: 12
Number of graphs: 142
Adjacency matrix shape: (7, 7)


By loading the DGL dataset, accessing the first graph, and retrieving information such as the number of vertices, edges, and the shape of the adjacency matrix, the code provides insights into the structure of the graph stored in the dataset.

In [63]:
# Get the dataset
dataset = DGLDatasetClass(address=path_data_temp + "_test")

# Get the first graph in the dataset
graph = dataset[0][0]

# Get the number of vertices, edges, and graphs
num_vertices = graph.number_of_nodes()
num_edges = graph.number_of_edges()
num_graphs = len(dataset)

print("Number of vertices:", num_vertices)
print("Number of edges:", num_edges)
print("Number of graphs:", num_graphs)

# Get the adjacency matrix shape
adj_matrix = graph.adjacency_matrix()
adj_shape = adj_matrix.shape

print("Adjacency matrix shape:", adj_shape)


Number of vertices: 29
Number of edges: 58
Number of graphs: 144
Adjacency matrix shape: (29, 29)


The code accesses the global features from the train, validation, and test sets and prints them. Here's how it works:

1. `train_globals = [globals for _, _, _, globals in train_set]`: This line retrieves the global features from the train set and assigns them to the `train_globals` list. Each global feature corresponds to a molecule in the train set.
2. `val_globals = [globals for _, _, _, globals in val_set]`: This line retrieves the global features from the validation set and assigns them to the `val_globals` list. Each global feature corresponds to a molecule in the validation set.
3. `test_globals = [globals for _, _, _, globals in test_set]`: This line retrieves the global features from the test set and assigns them to the `test_globals` list. Each global feature corresponds to a molecule in the test set.

4. The following lines print the global features:
   - `print("Global Features for Train Set:")`: Prints a header indicating the train set global features are being displayed.
   - `for i, globals in enumerate(train_globals):`: Iterates over the train_globals list.
     - `print(f"Molecule {i+1}: {globals}")`: Prints the global features for each molecule in the train set.
   - `print()`: Prints an empty line for separation.
   - `print("Global Features for Validation Set:")`: Prints a header indicating the validation set global features are being displayed.
   - `for i, globals in enumerate(val_globals):`: Iterates over the val_globals list.
     - `print(f"Molecule {i+1}: {globals}")`: Prints the global features for each molecule in the validation set.
   - `print()`: Prints an empty line for separation.
   - `print("Global Features for Test Set:")`: Prints a header indicating the test set global features are being displayed.
   - `for i, globals in enumerate(test_globals):`: Iterates over the test_globals list.
     - `print(f"Molecule {i+1}: {globals}")`: Prints the global features for each molecule in the test set.

By accessing the global features using the tuple unpacking syntax (`for _, _, _, globals in ...`), the code retrieves and displays the global features for each molecule in the train, validation, and test sets.

In [64]:
# Access the global features
train_globals = [globals for _, _, _, globals in train_set]
val_globals = [globals for _, _, _, globals in val_set]
test_globals = [globals for _, _, _, globals in test_set]

# Print the global features
print("Global Features for Train Set:")
for i, globals in enumerate(train_globals):
    print(f"Molecule {i+1}: {globals}")
print()

print("Global Features for Validation Set:")
for i, globals in enumerate(val_globals):
    print(f"Molecule {i+1}: {globals}")
print()

print("Global Features for Test Set:")
for i, globals in enumerate(test_globals):
    print(f"Molecule {i+1}: {globals}")
print()


Global Features for Train Set:
Molecule 1: tensor([9.5933e-01, 1.9450e-03, 1.6251e-02, 2.9441e-02, 1.9749e-02, 1.7600e-02,
        4.2251e-02, 2.6595e-02, 1.6925e-02, 8.6182e-03, 1.3757e-02, 7.3703e-03,
        1.3908e-02, 7.8459e-03, 4.0749e-10, 2.1752e-08, 6.8936e-07, 1.0022e-01,
        3.6384e-01, 8.1139e-01, 9.2584e-11, 5.8561e-17, 1.0887e-06, 4.8816e-01,
        6.4639e-01, 1.3135e-02, 6.8859e-03, 4.8852e-03, 3.7190e-03, 9.9939e-01,
        9.8356e-01, 1.4189e-02, 9.1647e-03, 1.0000e+00, 9.7520e-02, 8.6029e-01,
        9.7224e-01, 1.5339e-02, 1.5957e-02, 1.3223e-01, 1.5957e-02, 9.2554e-03,
        9.8420e-01, 5.2792e-03, 8.8089e-01, 8.5824e-01, 5.9319e-03, 2.7953e-02,
        1.2869e-02, 9.7629e-01, 3.0517e-01, 9.9386e-22, 1.4249e-01, 6.2963e-02,
        3.4723e-02, 4.8299e-15, 1.1178e-02, 4.9226e-01, 9.5102e-01, 2.0751e-01,
        5.6952e-08, 8.3334e-01, 0.0000e+00, 1.1424e-21, 2.4025e-23, 7.1394e-02,
        9.1444e-01, 5.7614e-21, 3.6088e-15, 1.4598e-01, 1.7356e-22, 1.1809e-1

Exception in thread _colab_inspector_thread:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/google/colab/_debugpy.py", line 64, in inspector_thread
    _variable_inspector.run(shell, time)
  File "/usr/local/lib/python3.10/dist-packages/google/colab/_variable_inspector.py", line 27, in run
    globals().clear()
TypeError: 'Tensor' object is not callable


Streaming output truncated to the last 5000 lines.
        9.9844e-01, 9.9858e-01, 9.9761e-01, 9.9675e-01, 9.9604e-01, 9.9443e-01,
        9.9340e-01, 9.9095e-01, 9.8271e-01, 9.9972e-01, 6.8936e-07, 9.9830e-01,
        6.7984e-01, 9.7929e-01, 9.2843e-01, 5.8561e-17, 1.0887e-06, 9.3780e-01,
        9.4126e-01, 9.9896e-01, 2.3071e-02, 3.0565e-02, 3.7034e-02, 9.5607e-01,
        4.3663e-03, 9.9880e-01, 9.9887e-01, 1.0000e+00, 9.9933e-01, 9.9936e-01,
        1.0000e+00, 9.9880e-01, 9.0717e-01, 9.5679e-01, 9.0717e-01, 7.4290e-01,
        1.7488e-01, 7.3917e-01, 7.3013e-02, 1.9678e-02, 3.4752e-03, 9.9849e-01,
        9.9916e-01, 9.9906e-01, 9.9998e-01, 9.9386e-22, 9.7155e-01, 9.6456e-01,
        3.5821e-01, 4.8299e-15, 8.3758e-02, 9.9999e-01, 9.9939e-01, 9.9990e-01,
        5.6952e-08, 9.9220e-01, 0.0000e+00, 9.9907e-01, 9.6593e-01, 9.9889e-01,
        9.9925e-01, 9.9951e-01, 3.6088e-15, 9.7510e-01, 9.9412e-01, 1.1809e-10,
        9.9963e-01, 9.0550e-08, 4.6098e-10, 1.5707e-01, 9.5188e-01, 9

The code accesses the output masks for the train, validation, and test sets, and then prints the masks.As you can see, there is no empty cell and values of 1 are given.

In [65]:
# Access the output masks
train_masks = train_set.masks
val_masks = val_set.masks
test_masks = test_set.masks

# Print the output masks
print("Train Set Output Masks:")
print(train_masks)
print()

print("Validation Set Output Masks:")
print(val_masks)
print()

print("Test Set Output Masks:")
print(test_masks)


Train Set Output Masks:
tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]])

Validation Set Output Masks:
tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]])

Test Set Output Masks:
tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]])


This code checks if there are any empty cells (cells with NaN values) in the train, validation, and test sets.
By iterating over the samples and checking for NaN values in the labels tensor, the code determines if there are any empty cells present in the train, validation, and test sets.

In [66]:
# Check if there is an empty cell in the train_set
empty_cells_train = any(torch.isnan(labels).any() for _, labels, _, _ in train_set)
if empty_cells_train:
    print("Train set contains empty cells.")
else:
    print("Train set does not contain empty cells.")

# Check if there is an empty cell in the val_set
empty_cells_val = any(torch.isnan(labels).any() for _, labels, _, _ in val_set)
if empty_cells_val:
    print("Validation set contains empty cells.")
else:
    print("Validation set does not contain empty cells.")

# Check if there is an empty cell in the test_set
empty_cells_test = any(torch.isnan(labels).any() for _, labels, _, _ in test_set)
if empty_cells_test:
    print("Test set contains empty cells.")
else:
    print("Test set does not contain empty cells.")


Train set does not contain empty cells.
Validation set does not contain empty cells.
Test set does not contain empty cells.


Because the data is a task, there is no empty cell

In [67]:
# Data Loader
def collate(batch):
    graphs = [e[0] for e in batch]
    g = dgl.batch(graphs)

    labels = [e[1] for e in batch]
    labels = torch.stack(labels, 0)

    masks = [e[2] for e in batch]
    masks = torch.stack(masks, 0)

    globals = [e[3] for e in batch]
    globals = torch.stack(globals, 0)

    return g, labels, masks, globals

def loader(batch_size=64):
    train_dataloader = DataLoader(train_set,
                                  batch_size=batch_size,
                                  collate_fn=collate,
                                  drop_last=False,
                                  shuffle=True,
                                  num_workers=1)

    val_dataloader = DataLoader(val_set,
                                batch_size=batch_size,
                                collate_fn=collate,
                                drop_last=False,
                                shuffle=False,
                                num_workers=1)

    test_dataloader = DataLoader(test_set,
                                 batch_size=batch_size,
                                 collate_fn=collate,
                                 drop_last=False,
                                 shuffle=False,
                                 num_workers=1)

    return train_dataloader, val_dataloader, test_dataloader


In [68]:

train_dataloader, val_dataloader, test_dataloader = loader(batch_size=64)

In [69]:
g,_,_,_=next(iter(train_dataloader))
g

Graph(num_nodes=517, num_edges=916,
      ndata_schemes={'v': Scheme(shape=(128,), dtype=torch.float32)}
      edata_schemes={'e': Scheme(shape=(13,), dtype=torch.float32)})

In [70]:
# Set the number of tasks in the SIDER dataset
num_tasks = 27

# Set the size of the global features based on the number of nodes in the molecular graph (MG)
global_size = 48006

# Number of epochs to train the model
num_epochs = 100

# Number of steps to wait if the model performance on the validation set does not improve
patience = 10
config = {"node_feature_size": 127, "edge_feature_size": 12, "hidden_size": 100}


In [71]:
# GNN Model
class GNN(nn.Module):
    def __init__(self, config, global_size=200, num_tasks=1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        self.node_feature_size = self.config.get('node_feature_size', 127)
        self.edge_feature_size = self.config.get('edge_feature_size', 12)
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = GraphConv(self.node_feature_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv2 = GraphConv(self.hidden_size, self.num_tasks, allow_zero_in_degree=True)

    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:, :self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:, :self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")


In [72]:
from sklearn.metrics import roc_auc_score
# Compute Score Function
def compute_score(model, data_loader, val_size, num_tasks):
    model.eval()
    metric = roc_auc_score
    with torch.no_grad():
        prediction_all = torch.empty(0)
        labels_all = torch.empty(0)
        masks_all = torch.empty(0)
        for i, (mol_dgl_graph, labels, masks, globals) in enumerate(data_loader):
            prediction = model(mol_dgl_graph, globals)
            prediction = torch.sigmoid(prediction)
            prediction_all = torch.cat((prediction_all, prediction), 0)
            labels_all = torch.cat((labels_all, labels), 0)
            masks_all = torch.cat((masks_all, masks), 0)
        average = torch.tensor([0.])
        for i in range(num_tasks):
            a1 = prediction_all[:, i][masks_all[:, i] == 1]
            a2 = labels_all[:, i][masks_all[:, i] == 1]
            try:
                t = metric(a2.int().cpu(), a1.cpu()).item()
            except ValueError:
                t = 0
            average += t
    return average.item() / num_tasks

In [73]:
# Loss Function
def loss_func(output, label, mask, num_tasks):
    pos_weight = torch.ones((1, num_tasks))
    criterion = torch.nn.BCEWithLogitsLoss(reduction='none', pos_weight=pos_weight)
    loss = mask * criterion(output, label)
    loss = loss.sum() / mask.sum()
    return loss


In [74]:
# Training Function
def train_epoch(train_dataloader, model, optimizer):
    epoch_train_loss = 0
    iterations = 0
    model.train()
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(train_dataloader):
        prediction = model(mol_dgl_graph, globals)
        loss_train = loss_func(prediction, labels, masks, num_tasks)
        optimizer.zero_grad(set_to_none=True)
        loss_train.backward()
        optimizer.step()
        epoch_train_loss += loss_train.detach().item()
        iterations += 1
    epoch_train_loss /= iterations
    return epoch_train_loss

In [75]:
# Train and Evaluate Function
def train_evaluate():
    model = GNN(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(),lr=0.0001)

    best_val = 0
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)
            if score_val > best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
                epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # Best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")


In [76]:
# Test Evaluation Function
def test_evaluate():
    final_model = GNN(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

In [77]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.700 | Valid Score: 0.519
 
Epoch: 1/100 | Best Valid Score Until Now: 0.519 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.689 | Valid Score: 0.527
 
Epoch: 2/100 | Best Valid Score Until Now: 0.527 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.678 | Valid Score: 0.536
 
Epoch: 3/100 | Best Valid Score Until Now: 0.536 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.668 | Valid Score: 0.543
 
Epoch: 4/100 | Best Valid Score Until Now: 0.543 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.658 | Valid Score: 0.550
 
Epoch: 5/100 | Best Valid Score Until Now: 0.550 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.648 | Valid Score: 0.553
 
Epoch: 6/100 | Best Valid Score Until Now: 0.553 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.638 | Valid Score: 0.554
 
Epoch: 7/100 | Best Valid Score Until Now: 0.554 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.627 | Valid Score: 0.555
 
Epoch: 8/100 | Best Valid Score Until Now: 

#sage

##message copy and reduce mean

This code defines a custom SAGEConv (GraphSAGE Convolution) module in PyTorch for graph convolutional operations.
The `SAGEConv` class extends the `nn.Module` class and initializes a SAGEConv layer. It takes the input feature size (`in_feat`), output feature size (`out_feat`), and aggregator type as arguments. The aggregator type determines how neighbor information is aggregated, with `'mean'` being the default.

The `forward` method performs the graph convolution operation. It takes a DGL graph (`g`) and a tensor of node features (`h`) as inputs. The method begins by setting the node features in the graph with `g.ndata["h"] = h`. Then, it performs message passing using `update_all`. The `message_func` copies the node features to messages, while the `reduce_func` applies the specified aggregator type to aggregate the messages from neighboring nodes.
After message passing, the method retrieves the aggregated node features (`h_N`) from the graph. It concatenates the original node features (`h`) with the aggregated features (`h_N`). Finally, the concatenated features are passed through a linear transformation (`self.linear`) to obtain the output features.

In [31]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl
from dgl.nn import GraphConv
import dgl.function as fn
from dgl.nn import SAGEConv


class SAGEConv(nn.Module):

    def __init__(self, in_feat, out_feat, aggregator_type='mean'):
        super(SAGEConv, self).__init__()
        self.aggregator_type = aggregator_type
        self.linear = nn.Linear(in_feat * 2, out_feat)

    def forward(self, g, h):
        with g.local_scope():
            g.ndata["h"] = h
            g.update_all(
                message_func=fn.copy_u("h", "m"),
                reduce_func=getattr(fn, self.aggregator_type)("m", "h_N"),
            )
            h_N = g.ndata["h_N"]
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)

this code defines a GNN model that performs graph convolution operations using the SAGEConv module. It applies two SAGEConv layers to obtain node representations and aggregates them to compute the final output of the model.

In [32]:
class GNN(nn.Module):
    def __init__(self, config, global_size=200, num_tasks=1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv(self.node_feature_size, self.hidden_size)
        self.conv2 = SAGEConv(self.hidden_size, self.num_tasks)

    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:, :self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:, :self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")


In [33]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.693 | Valid Score: 0.531
 
Epoch: 1/100 | Best Valid Score Until Now: 0.531 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.681 | Valid Score: 0.538
 
Epoch: 2/100 | Best Valid Score Until Now: 0.538 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.667 | Valid Score: 0.542
 
Epoch: 3/100 | Best Valid Score Until Now: 0.542 

Patience 1
Epoch: 4/100 | Training Loss: 0.651 | Valid Score: 0.542
 
Epoch: 4/100 | Best Valid Score Until Now: 0.542 

Patience 2
Epoch: 5/100 | Training Loss: 0.632 | Valid Score: 0.542
 
Epoch: 5/100 | Best Valid Score Until Now: 0.542 

Patience 3
Epoch: 6/100 | Training Loss: 0.610 | Valid Score: 0.541
 
Epoch: 6/100 | Best Valid Score Until Now: 0.542 

Patience 4
Epoch: 7/100 | Training Loss: 0.590 | Valid Score: 0.542
 
Epoch: 7/100 | Best Valid Score Until Now: 0.542 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.573 | Valid Score: 0.543
 
Epoch: 8/100 | Best Valid Score Until Now: 0.543 

Save checkpo

##message add and reduce sum

The `SAGEConv1` class extends the `nn.Module` class and defines a SAGEConv convolutional layer. `SAGEConv1` applies a linear transformation to the concatenation of the node features and the sum of their neighbors' features.

The `forward` method of `SAGEConv1` takes a DGL graph `g` and a tensor of node features `h` as inputs. It first sets the node features in the graph `g`, then performs message passing using the `u_add_v` and `sum` built-in DGL functions to compute the sum of the neighbor features for each node. Finally, it concatenates the original node features with the sum of neighbor features, applies a linear transformation, and returns the result.

In [34]:
class SAGEConv1(nn.Module):

    def __init__(self, in_feat, out_feat):
        super(SAGEConv1, self).__init__()

        self.linear = nn.Linear(in_feat * 2, out_feat)

        #The forward method of SAGEConv1 takes a DGL graph g and a tensor of node features h as inputs.
        # It first sets the node features in the graph g, then performs message passing using the u_add_v and sum built-in DGL functions to compute the sum of the neighbor features for each node.
        # Finally, it concatenates the original node features with the sum of neighbor features, applies a linear transformation, and returns the result.


    def forward(self, g, h):
        with g.local_scope():
            g.ndata["h"] = h
            g.update_all(
                message_func=fn.u_add_v("h","h", "m"),
                reduce_func=fn.sum("m", "h_N"),
            )
            h_N = g.ndata["h_N"]
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)


The `GNN` class extends the `nn.Module` class and initializes a GNN model. It takes a configuration dictionary (`config`), global size, and number of tasks as arguments. The model has two SAGEConv1 layers (`self.conv1` and `self.conv2`) for performing graph convolution operations.
The `forward` method performs the forward pass of the GNN model. It takes a DGL graph (`mol_dgl_graph`) and global features (`globals`) as inputs. The method first restricts the node features and edge features to their respective sizes by slicing the tensors.

Then, it applies the first SAGEConv1 layer (`self.conv1`) to the node features, followed by a ReLU activation function (`F.relu`). Next, it applies the second SAGEConv1 layer (`self.conv2`) to obtain the final node representations.

The final node representations are stored in the graph with `mol_dgl_graph.ndata["h"] = h`. Finally, the method computes the mean of the node representations using `dgl.mean_nodes` with the feature name "h" and returns the result.



In [35]:
class GNN(nn.Module):
    def __init__(self, config, global_size=200, num_tasks=27):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv1(self.node_feature_size, self.hidden_size)
        self.conv2 = SAGEConv1(self.hidden_size, self.num_tasks)

    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:, :self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:, :self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")


In [36]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.739 | Valid Score: 0.510
 
Epoch: 1/100 | Best Valid Score Until Now: 0.510 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.675 | Valid Score: 0.518
 
Epoch: 2/100 | Best Valid Score Until Now: 0.518 

Patience 1
Epoch: 3/100 | Training Loss: 0.626 | Valid Score: 0.513
 
Epoch: 3/100 | Best Valid Score Until Now: 0.518 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.587 | Valid Score: 0.521
 
Epoch: 4/100 | Best Valid Score Until Now: 0.521 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.561 | Valid Score: 0.529
 
Epoch: 5/100 | Best Valid Score Until Now: 0.529 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.544 | Valid Score: 0.533
 
Epoch: 6/100 | Best Valid Score Until Now: 0.533 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.536 | Valid Score: 0.534
 
Epoch: 7/100 | Best Valid Score Until Now: 0.534 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.531 | Valid Score: 0.535
 
Epoch: 8/100 | Best Valid Score Until Now: 0.535

##message add and reduce sum

In [37]:
class SAGEConv2(nn.Module):
    def __init__(self, in_feat, out_feat):
        super(SAGEConv2, self).__init__()
        self.linear = nn.Linear(in_feat * 2, out_feat)

    def forward(self, g, h):
        with g.local_scope():
            g.ndata["h"] = h
            g.send_and_recv(g.edges(), fn.u_add_v("h", "h", "m"), fn.sum("m", "h_N"))

            h_N = g.ndata["h_N"]
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)

class GNN(nn.Module):
    def __init__(self, config, global_size=200, num_tasks=1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv2(self.node_feature_size, self.hidden_size)
        self.conv2 = SAGEConv2(self.hidden_size, self.num_tasks)
#The forward method of SAGEConv2 takes a DGL graph g and a tensor of node features h as inputs.
# It first sets the node features in the graph g, then performs message passing and aggregation using the send_and_recv function.
#It sends messages by applying the "add" message function fn.u_add_v("h", "h", "m") to each edge of the graph.
#The messages are then received and aggregated using the "sum" reduce function fn.sum("m", "h_N") to compute the sum of the neighbor features for each node.
#After the message passing and aggregation step, it retrieves the aggregated neighbor features h_N from the graph data. It concatenates the original node features h with the aggregated neighbor features h_N, applies a linear transformation, and returns the result.
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:, :self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:, :self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

#

start_time = time.time()

train_evaluate()
test_evaluate()


Save checkpoint
Epoch: 1/100 | Training Loss: 0.726 | Valid Score: 0.489
 
Epoch: 1/100 | Best Valid Score Until Now: 0.489 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.662 | Valid Score: 0.515
 
Epoch: 2/100 | Best Valid Score Until Now: 0.515 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.612 | Valid Score: 0.529
 
Epoch: 3/100 | Best Valid Score Until Now: 0.529 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.572 | Valid Score: 0.531
 
Epoch: 4/100 | Best Valid Score Until Now: 0.531 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.548 | Valid Score: 0.533
 
Epoch: 5/100 | Best Valid Score Until Now: 0.533 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.537 | Valid Score: 0.535
 
Epoch: 6/100 | Best Valid Score Until Now: 0.535 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.532 | Valid Score: 0.537
 
Epoch: 7/100 | Best Valid Score Until Now: 0.537 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.530 | Valid Score: 0.539
 
Epoch: 8/100 | Best Valid Score Until Now: 

##message div and reduce max

this code defines a SAGEConv3 module for graph convolutional operations, which uses "div" as the message function to compute element-wise division of node features and "mean" as the reduce function to compute the mean of the aggregated messages from neighboring nodes.

In [38]:
class SAGEConv3(nn.Module):

    def __init__(self, in_feat, out_feat):
        super(SAGEConv3, self).__init__()

        self.linear = nn.Linear(in_feat * 2, out_feat)

    def forward(self, g, h):
        with g.local_scope():
            g.ndata["h"] = h
            g.update_all(
                message_func=fn.u_div_v("h","h", "m"),
                reduce_func=fn.mean("m", "h_N"),
            )
            h_N = g.ndata["h_N"]
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)

In [39]:
class GNN(nn.Module):
    def __init__(self, config, global_size=200, num_tasks=27):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv3(self.node_feature_size, self.hidden_size)
        self.conv2 = SAGEConv3(self.hidden_size, self.num_tasks)

    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:, :self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:, :self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")


In [40]:

start_time = time.time()

train_evaluate()
test_evaluate()


Patience 1
Epoch: 1/100 | Training Loss: nan | Valid Score: 0.000
 
Epoch: 1/100 | Best Valid Score Until Now: 0.000 

Patience 2
Epoch: 2/100 | Training Loss: nan | Valid Score: 0.000
 
Epoch: 2/100 | Best Valid Score Until Now: 0.000 

Patience 3
Epoch: 3/100 | Training Loss: nan | Valid Score: 0.000
 
Epoch: 3/100 | Best Valid Score Until Now: 0.000 

Patience 4
Epoch: 4/100 | Training Loss: nan | Valid Score: 0.000
 
Epoch: 4/100 | Best Valid Score Until Now: 0.000 

Patience 5
Epoch: 5/100 | Training Loss: nan | Valid Score: 0.000
 
Epoch: 5/100 | Best Valid Score Until Now: 0.000 

Patience 6
Epoch: 6/100 | Training Loss: nan | Valid Score: 0.000
 
Epoch: 6/100 | Best Valid Score Until Now: 0.000 

Patience 7
Epoch: 7/100 | Training Loss: nan | Valid Score: 0.000
 
Epoch: 7/100 | Best Valid Score Until Now: 0.000 

Patience 8
Epoch: 8/100 | Training Loss: nan | Valid Score: 0.000
 
Epoch: 8/100 | Best Valid Score Until Now: 0.000 

Patience 9
Epoch: 9/100 | Training Loss: nan | V

##message sub and reduce mean

this code defines a SAGEConv4 module for graph convolutional operations, which uses "sub" as the message function to compute element-wise subtraction of node features and "mean" as the reduce function to compute the mean of the aggregated messages from neighboring nodes.

In [41]:
class SAGEConv4(nn.Module):

    def __init__(self, in_feat, out_feat):
        super(SAGEConv4, self).__init__()

        self.linear = nn.Linear(in_feat * 2, out_feat)
#The forward method of SAGEConv4 takes a DGL graph g and a tensor of node features h as inputs. It first sets the node features in the graph g, then performs message passing and aggregation using the update_all function. It sends messages by applying the "sub" message function fn.u_sub_v("h", "h", "m") to each edge of the graph. The messages are then received and aggregated using the "mean" reduce function fn.mean("m", "h_N") to compute the mean of the neighbor features for each node.

    def forward(self, g, h):
        with g.local_scope():
            g.ndata["h"] = h
            g.update_all(
                message_func = fn.u_sub_v('h', 'h', 'm'),
                reduce_func=fn.mean("m", "h_N"),
            )
            h_N = g.ndata["h_N"]
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)

In [42]:
class GNN(nn.Module):
    def __init__(self, config, global_size=200, num_tasks=1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv4(self.node_feature_size, self.hidden_size)
        self.conv2 = SAGEConv4(self.hidden_size, self.num_tasks)

    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:, :self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:, :self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")


In [43]:

start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.693 | Valid Score: 0.522
 
Epoch: 1/100 | Best Valid Score Until Now: 0.522 

Patience 1
Epoch: 2/100 | Training Loss: 0.686 | Valid Score: 0.522
 
Epoch: 2/100 | Best Valid Score Until Now: 0.522 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.679 | Valid Score: 0.524
 
Epoch: 3/100 | Best Valid Score Until Now: 0.524 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.671 | Valid Score: 0.527
 
Epoch: 4/100 | Best Valid Score Until Now: 0.527 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.662 | Valid Score: 0.530
 
Epoch: 5/100 | Best Valid Score Until Now: 0.530 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.651 | Valid Score: 0.534
 
Epoch: 6/100 | Best Valid Score Until Now: 0.534 

Patience 1
Epoch: 7/100 | Training Loss: 0.639 | Valid Score: 0.533
 
Epoch: 7/100 | Best Valid Score Until Now: 0.534 

Patience 2
Epoch: 8/100 | Training Loss: 0.627 | Valid Score: 0.532
 
Epoch: 8/100 | Best Valid Score Until Now: 0.534 

Patienc

##message mul and reduce mean

 this code defines a SAGEConv5 module for graph convolutional operations, which uses "mul" as the message function to compute element-wise multiplication of node features and "mean" as the reduce function to compute the mean of the aggregated messages from neighboring nodes.

In [44]:
class SAGEConv5(nn.Module):

    def __init__(self, in_feat, out_feat):
        super(SAGEConv5, self).__init__()

        self.linear = nn.Linear(in_feat * 2, out_feat)
#It performs message forwarding and aggregation using the ``update_all'' function. Sends messages by applying the "mul" message function "fn.u_mul_v("h", "h", "m")" to each edge of the graph. The messages are then received and summed using the mean reduction function "fn.mean("m", "h_N")" to calculate the average of the element-wise multiplied messages from the neighboring nodes.
    def forward(self, g, h):
        with g.local_scope():
            g.ndata["h"] = h
            g.update_all(
                message_func = fn.u_mul_v('h', 'h', 'm'),
                reduce_func=fn.mean("m", "h_N"),
            )
            h_N = g.ndata["h_N"]
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)

In [45]:
class GNN(nn.Module):
    def __init__(self, config, global_size=200, num_tasks=1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv5(self.node_feature_size, self.hidden_size)
        self.conv2 = SAGEConv5(self.hidden_size, self.num_tasks)

    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:, :self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:, :self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")


In [46]:
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.691 | Valid Score: 0.521
 
Epoch: 1/100 | Best Valid Score Until Now: 0.521 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.683 | Valid Score: 0.533
 
Epoch: 2/100 | Best Valid Score Until Now: 0.533 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.674 | Valid Score: 0.538
 
Epoch: 3/100 | Best Valid Score Until Now: 0.538 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.663 | Valid Score: 0.545
 
Epoch: 4/100 | Best Valid Score Until Now: 0.545 

Patience 1
Epoch: 5/100 | Training Loss: 0.651 | Valid Score: 0.542
 
Epoch: 5/100 | Best Valid Score Until Now: 0.545 

Patience 2
Epoch: 6/100 | Training Loss: 0.635 | Valid Score: 0.542
 
Epoch: 6/100 | Best Valid Score Until Now: 0.545 

Patience 3
Epoch: 7/100 | Training Loss: 0.618 | Valid Score: 0.541
 
Epoch: 7/100 | Best Valid Score Until Now: 0.545 

Patience 4
Epoch: 8/100 | Training Loss: 0.601 | Valid Score: 0.541
 
Epoch: 8/100 | Best Valid Score Until Now: 0.545 

Patience 5
E

##message mul and reduce sum

 this code defines a SAGEConv6 module for graph convolutional operations, which uses "mul" as the message function to compute element-wise multiplication of node features and "sum" as the reduce function to compute the sum of the aggregated messages from neighboring nodes.

In [47]:
class SAGEConv6(nn.Module):

    def __init__(self, in_feat, out_feat):
        super(SAGEConv6, self).__init__()

        self.linear = nn.Linear(in_feat * 2, out_feat)

    def forward(self, g, h):
        with g.local_scope():
            g.ndata["h"] = h
            g.update_all(
                message_func = fn.u_mul_v('h', 'h', 'm'),
                reduce_func=fn.sum("m", "h_N"),
            )
            h_N = g.ndata["h_N"]
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)

In [48]:
class GNN(nn.Module):
    def __init__(self, config, global_size=200, num_tasks=1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv6(self.node_feature_size, self.hidden_size)
        self.conv2 = SAGEConv6(self.hidden_size, self.num_tasks)

    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:, :self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:, :self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")


In [49]:
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.702 | Valid Score: 0.494
 
Epoch: 1/100 | Best Valid Score Until Now: 0.494 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.685 | Valid Score: 0.501
 
Epoch: 2/100 | Best Valid Score Until Now: 0.501 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.668 | Valid Score: 0.520
 
Epoch: 3/100 | Best Valid Score Until Now: 0.520 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.651 | Valid Score: 0.529
 
Epoch: 4/100 | Best Valid Score Until Now: 0.529 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.633 | Valid Score: 0.532
 
Epoch: 5/100 | Best Valid Score Until Now: 0.532 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.613 | Valid Score: 0.533
 
Epoch: 6/100 | Best Valid Score Until Now: 0.533 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.595 | Valid Score: 0.535
 
Epoch: 7/100 | Best Valid Score Until Now: 0.535 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.580 | Valid Score: 0.536
 
Epoch: 8/100 | Best Valid Score Until Now: 

##message mul and reduce min

, the code defines a new `SAGEConv7` module for graph convolutional operations. This module uses "mul" as the message function to perform element-wise multiplication between node features, and "min" as the reduce function to compute the minimum of the aggregated messages from neighboring nodes.


In [50]:
dgl.use_libxsmm(False)  # Disable libxsmm

class SAGEConv7(nn.Module):
    def __init__(self, in_feat, out_feat):
        super(SAGEConv7, self).__init__()
        self.linear = nn.Linear(in_feat * 2, out_feat)

    def forward(self, g, h):
        with g.local_scope():
            g.ndata["h"] = h
            g.update_all(
                message_func=fn.u_mul_v('h', 'h', 'm'),
                reduce_func=fn.min("m", "h_N"),
            )
            h_N = g.ndata["h_N"]
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)

class GNN(nn.Module):
    def __init__(self, config, global_size=200, num_tasks=1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv7(self.node_feature_size, self.hidden_size)
        self.conv2 = SAGEConv7(self.hidden_size, self.num_tasks)

    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:, :self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:, :self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

start_time = time.time()

train_evaluate()
test_evaluate()


Save checkpoint
Epoch: 1/100 | Training Loss: 0.694 | Valid Score: 0.496
 
Epoch: 1/100 | Best Valid Score Until Now: 0.496 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.688 | Valid Score: 0.512
 
Epoch: 2/100 | Best Valid Score Until Now: 0.512 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.681 | Valid Score: 0.518
 
Epoch: 3/100 | Best Valid Score Until Now: 0.518 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.672 | Valid Score: 0.526
 
Epoch: 4/100 | Best Valid Score Until Now: 0.526 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.661 | Valid Score: 0.528
 
Epoch: 5/100 | Best Valid Score Until Now: 0.528 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.646 | Valid Score: 0.530
 
Epoch: 6/100 | Best Valid Score Until Now: 0.530 

Patience 1
Epoch: 7/100 | Training Loss: 0.630 | Valid Score: 0.529
 
Epoch: 7/100 | Best Valid Score Until Now: 0.530 

Patience 2
Epoch: 8/100 | Training Loss: 0.613 | Valid Score: 0.530
 
Epoch: 8/100 | Best Valid Score Until Now: 0.530 

Pa

##message mul and reduce max

The `forward` method of `SAGEConv8` is similar to the previous SAGEConv layers. It sets the node features in the graph `g`, performs message passing using "mul" as the message function, and aggregation using "max" as the reduce function. It then retrieves the aggregated neighbor features `h_N` from the graph data, concatenates the original node features `h` with the aggregated neighbor features `h_N`, applies a linear transformation, and returns the result.

In [51]:
class SAGEConv8(nn.Module):

    def __init__(self, in_feat, out_feat):
        super(SAGEConv8, self).__init__()

        self.linear = nn.Linear(in_feat * 2, out_feat)

    def forward(self, g, h):
        with g.local_scope():
            g.ndata["h"] = h
            g.update_all(
                message_func = fn.u_mul_v('h', 'h', 'm'),
                reduce_func=fn.max("m", "h_N"),
            )
            h_N = g.ndata["h_N"]
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)

In [52]:
class GNN(nn.Module):
    def __init__(self, config, global_size=200, num_tasks=1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv8(self.node_feature_size, self.hidden_size)
        self.conv2 = SAGEConv8(self.hidden_size, self.num_tasks)

    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:, :self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:, :self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")


In [53]:
start_time = time.time()

train_evaluate()
test_evaluate()


Save checkpoint
Epoch: 1/100 | Training Loss: 0.695 | Valid Score: 0.494
 
Epoch: 1/100 | Best Valid Score Until Now: 0.494 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.688 | Valid Score: 0.501
 
Epoch: 2/100 | Best Valid Score Until Now: 0.501 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.681 | Valid Score: 0.512
 
Epoch: 3/100 | Best Valid Score Until Now: 0.512 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.672 | Valid Score: 0.524
 
Epoch: 4/100 | Best Valid Score Until Now: 0.524 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.659 | Valid Score: 0.530
 
Epoch: 5/100 | Best Valid Score Until Now: 0.530 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.644 | Valid Score: 0.535
 
Epoch: 6/100 | Best Valid Score Until Now: 0.535 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.625 | Valid Score: 0.539
 
Epoch: 7/100 | Best Valid Score Until Now: 0.539 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.606 | Valid Score: 0.540
 
Epoch: 8/100 | Best Valid Score Until Now: 

##add layers

`SAGEConv9`: This class extends the `nn.Module` base class from PyTorch and defines a custom graph convolutional layer using the SAGE (GraphSAGE) algorithm. It takes the input feature size `in_feat` and output feature size `out_feat` as arguments. The `forward` method performs the graph convolution operation using element-wise multiplication as the message function (`fn.u_mul_v`) and sum reduction (`fn.sum`) to aggregate messages from neighboring nodes. The resulting node features are concatenated with the original node features, passed through a linear transformation, and returned.

- `GNN`: This class extends the `nn.Module` base class and implements a Graph Neural Network model. It takes a `config` dictionary, `global_size`, and `num_tasks` as arguments. The class defines the architecture of the GNN model, which includes three instances of the `SAGEConv9` layer (`conv1`, `conv2`, `conv3`). It also includes batch normalization layers (`bn1`, `bn2`) between the convolutional layers. The `forward` method performs the forward pass of the model, applying the graph convolutional layers on the input graph `mol_dgl_graph`. It first truncates the node and edge features to the desired sizes, applies the convolutional layers with batch normalization and ReLU activation, and computes the mean of the resulting node features.
At first we work with 3 layers and in the next code we work with 4 layers

In [54]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl
from dgl.nn import GraphConv
import dgl.function as fn
from dgl.nn import SAGEConv
import time


class SAGEConv9(nn.Module):
    def __init__(self, in_feat, out_feat):
        super(SAGEConv9, self).__init__()
        self.linear = nn.Linear(in_feat * 2, out_feat)

    def forward(self, g, h):
        with g.local_scope():
            g.ndata["h"] = h
            g.update_all(
                message_func=fn.u_mul_v('h', 'h', 'm'),
                reduce_func=fn.sum("m", "h_N"),
            )
            h_N = g.ndata["h_N"]
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)


class GNN(nn.Module):
    def __init__(self, config, global_size=200, num_tasks=1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv9(self.node_feature_size, self.hidden_size)
        self.bn1 = nn.BatchNorm1d(self.hidden_size)  # Define batch normalization layer bn1
        self.conv2 = SAGEConv9(self.hidden_size, self.hidden_size)
        self.bn2 = nn.BatchNorm1d(self.hidden_size)  # Define batch normalization layer bn2
        self.conv3 = SAGEConv9(self.hidden_size, self.num_tasks)

    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:, :self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:, :self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = self.bn1(h)
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = self.bn2(h)
        h = F.relu(h)
        h = self.conv3(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

config = {}  # Replace with your config
gnn = GNN(config, num_tasks=1)  # Set num_tasks to 1 for binary classification

start_time = time.time()

train_evaluate()
test_evaluate()


Save checkpoint
Epoch: 1/100 | Training Loss: 0.775 | Valid Score: 0.515
 
Epoch: 1/100 | Best Valid Score Until Now: 0.515 

Patience 1
Epoch: 2/100 | Training Loss: 0.714 | Valid Score: 0.509
 
Epoch: 2/100 | Best Valid Score Until Now: 0.515 

Patience 2
Epoch: 3/100 | Training Loss: 0.680 | Valid Score: 0.513
 
Epoch: 3/100 | Best Valid Score Until Now: 0.515 

Patience 3
Epoch: 4/100 | Training Loss: 0.666 | Valid Score: 0.511
 
Epoch: 4/100 | Best Valid Score Until Now: 0.515 

Patience 4
Epoch: 5/100 | Training Loss: 0.650 | Valid Score: 0.509
 
Epoch: 5/100 | Best Valid Score Until Now: 0.515 

Patience 5
Epoch: 6/100 | Training Loss: 0.633 | Valid Score: 0.508
 
Epoch: 6/100 | Best Valid Score Until Now: 0.515 

Patience 6
Epoch: 7/100 | Training Loss: 0.621 | Valid Score: 0.506
 
Epoch: 7/100 | Best Valid Score Until Now: 0.515 

Patience 7
Epoch: 8/100 | Training Loss: 0.612 | Valid Score: 0.506
 
Epoch: 8/100 | Best Valid Score Until Now: 0.515 

Patience 8
Epoch: 9/100 | T

In [55]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl
from dgl.nn import GraphConv
import dgl.function as fn
from dgl.nn import SAGEConv
import time


class SAGEConv9(nn.Module):
    def __init__(self, in_feat, out_feat):
        super(SAGEConv9, self).__init__()
        self.linear = nn.Linear(in_feat * 2, out_feat)

    def forward(self, g, h):
        with g.local_scope():
            g.ndata["h"] = h
            g.update_all(
                message_func=fn.u_mul_v('h', 'h', 'm'),
                reduce_func=fn.sum("m", "h_N"),
            )
            h_N = g.ndata["h_N"]
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)


class GNN(nn.Module):
    def __init__(self, config, global_size=200, num_tasks=27):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv9(self.node_feature_size, self.hidden_size)
        self.bn1 = nn.BatchNorm1d(self.hidden_size)  # Define batch normalization layer bn1
        self.conv2 = SAGEConv9(self.hidden_size, self.hidden_size)
        self.bn2 = nn.BatchNorm1d(self.hidden_size)  # Define batch normalization layer bn2
        self.conv3 = SAGEConv9(self.hidden_size, self.hidden_size)
        self.bn3 = nn.BatchNorm1d(self.hidden_size)
        self.conv4 = SAGEConv9(self.hidden_size, self.num_tasks)


    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:, :self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:, :self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = self.bn1(h)
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = self.bn2(h)
        h = F.relu(h)
        h = self.conv3(mol_dgl_graph, h)
        h = self.bn3(h)
        h = F.relu(h)
        h = self.conv4(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

config = {}  # Replace with your config
gnn = GNN(config, num_tasks=27)  # Set num_tasks to 1 for binary classification

start_time = time.time()

train_evaluate()
test_evaluate()


Save checkpoint
Epoch: 1/100 | Training Loss: 0.893 | Valid Score: 0.506
 
Epoch: 1/100 | Best Valid Score Until Now: 0.506 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.817 | Valid Score: 0.526
 
Epoch: 2/100 | Best Valid Score Until Now: 0.526 

Patience 1
Epoch: 3/100 | Training Loss: 0.767 | Valid Score: 0.516
 
Epoch: 3/100 | Best Valid Score Until Now: 0.526 

Patience 2
Epoch: 4/100 | Training Loss: 0.733 | Valid Score: 0.513
 
Epoch: 4/100 | Best Valid Score Until Now: 0.526 

Patience 3
Epoch: 5/100 | Training Loss: 0.718 | Valid Score: 0.511
 
Epoch: 5/100 | Best Valid Score Until Now: 0.526 

Patience 4
Epoch: 6/100 | Training Loss: 0.711 | Valid Score: 0.502
 
Epoch: 6/100 | Best Valid Score Until Now: 0.526 

Patience 5
Epoch: 7/100 | Training Loss: 0.689 | Valid Score: 0.514
 
Epoch: 7/100 | Best Valid Score Until Now: 0.526 

Patience 6
Epoch: 8/100 | Training Loss: 0.685 | Valid Score: 0.512
 
Epoch: 8/100 | Best Valid Score Until Now: 0.526 

Patience 7
Epoch: 9/10

the SAGEConv10 class represents a graph convolutional layer that performs message passing using element-wise addition (u_add_v) and aggregation using the maximum operation (max). The resulting features are concatenated with the original node features and transformed by a linear layer to produce the output of the layer.

In [56]:
class SAGEConv10(nn.Module):
    def __init__(self, in_feat, out_feat):
        super(SAGEConv10, self).__init__()
        self.linear = nn.Linear(in_feat * 2, out_feat)

    def forward(self, g, h):
        with g.local_scope():
            g.ndata["h"] = h
            g.update_all(
                message_func=fn.u_add_v('h', 'h', 'm'),
                reduce_func=fn.max("m", "h_N"),
            )
            h_N = g.ndata["h_N"]
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)


class GNN(nn.Module):
    def __init__(self, config, global_size=200, num_tasks=27):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv10(self.node_feature_size, self.hidden_size)
        self.bn1 = nn.BatchNorm1d(self.hidden_size)  # Define batch normalization layer bn1
        self.conv2 = SAGEConv10(self.hidden_size, self.hidden_size)
        self.bn2 = nn.BatchNorm1d(self.hidden_size)  # Define batch normalization layer bn2
        self.conv3 = SAGEConv10(self.hidden_size, self.hidden_size)
        self.bn3 = nn.BatchNorm1d(self.hidden_size)
        self.conv4 = SAGEConv10(self.hidden_size, self.num_tasks)


    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:, :self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:, :self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = self.bn1(h)
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = self.bn2(h)
        h = F.relu(h)
        h = self.conv3(mol_dgl_graph, h)
        h = self.bn3(h)
        h = F.relu(h)
        h = self.conv4(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

config = {}  # Replace with your config
gnn = GNN(config, num_tasks=27)  # Set num_tasks to 1 for binary classification

start_time = time.time()

train_evaluate()
test_evaluate()


Save checkpoint
Epoch: 1/100 | Training Loss: 0.713 | Valid Score: 0.483
 
Epoch: 1/100 | Best Valid Score Until Now: 0.483 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.650 | Valid Score: 0.490
 
Epoch: 2/100 | Best Valid Score Until Now: 0.490 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.608 | Valid Score: 0.506
 
Epoch: 3/100 | Best Valid Score Until Now: 0.506 

Patience 1
Epoch: 4/100 | Training Loss: 0.578 | Valid Score: 0.503
 
Epoch: 4/100 | Best Valid Score Until Now: 0.506 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.557 | Valid Score: 0.508
 
Epoch: 5/100 | Best Valid Score Until Now: 0.508 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.541 | Valid Score: 0.513
 
Epoch: 6/100 | Best Valid Score Until Now: 0.513 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.529 | Valid Score: 0.514
 
Epoch: 7/100 | Best Valid Score Until Now: 0.514 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.521 | Valid Score: 0.522
 
Epoch: 8/100 | Best Valid Score Until Now: 0.522